In [1]:
#### Double commented means it was commented out but was on the cluster initially.

import sys,os,os.path
#sys.path.append("../../")   # cite IC from parent directory
##sys.path.append("/gluster/data/next/software/IC_satkill/")
#sys.path.append("/gluster/data/next/software/IC_sophronia/")
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
##sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
##os.environ['ICTDIR']='/gluster/data/next/software/IC_satkill/'
sys.path.append("../../")   # cite IC from parent directory
sys.path.append("/home/e78368jw/Documents/NEXT_CODE/next_misc/")
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

import time

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clrs
from matplotlib.lines import Line2D

from tqdm import tqdm

import IC.invisible_cities.core.core_functions                   as     coref
import IC.invisible_cities.io.dst_io                           as     dstio

from IC.invisible_cities.cities                 import beersheba as beerfun

from IC.invisible_cities.evm.event_model                          import HitCollection

from IC.invisible_cities.database.load_db       import DataSiPM

from IC.invisible_cities.evm.event_model        import Cluster, Hit
from IC.invisible_cities.types.ic_types         import xy
from IC.invisible_cities.reco.paolina_functions import voxelize_hits, drop_end_point_voxels, make_track_graphs, get_track_energy

from IC.invisible_cities.evm.event_model        import HitEnergy
from IC.invisible_cities.cities.beersheba          import DeconvolutionMode
from IC.invisible_cities.cities.beersheba          import CutType

from IC.invisible_cities.reco import hits_functions as hif

from IC.invisible_cities.reco.deconv_functions import deconvolve
from IC.invisible_cities.reco.deconv_functions import deconvolution_input
from IC.invisible_cities.reco.deconv_functions import InterpolationMethod

import IC.invisible_cities.io.mcinfo_io as mcio

from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.colors import Normalize


In [2]:
file_no = 30
file = 'sophronia_' + str(file_no) + '_208Tl.h5'
##file = f'/gluster/data/next/files/TOPOLOGY_John/HYPPOS_DATA_QTHR/Q_THR4/PORT_1a/sophronia/sophronia_' + str(file_no) + '_208Tl.h5'

In [28]:
MC_df = pd.read_hdf(file, 'MC/particles')

In [6]:
#display(MC_df)
evt_id = MC_df[MC_df.particle_name == 'e+'].event_id.to_numpy()

In [7]:
print(evt_id)

[290013 290031 290032 290038 290044 290048 290051 290055 290058 290073
 290078 290081 290083 290099 290106 290108 290110 290112 290114 290125
 290140 290155 290159 290166 290173 290178 290215 290216 290218 290230
 290253 290261 290273 290278 290292 290304 290310 290315 290337 290342
 290353]


In [8]:
pd.set_option('display.max_rows', 500)
display(MC_df[MC_df.event_id == evt_id[0]])

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
1044,290013,1,Tl208,True,0,396.107330,393.276245,534.039307,0.000000,396.107330,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,none,RadioactiveDecay
1045,290013,4,e-,False,1,396.107330,393.276245,534.039307,0.000183,404.240479,...,0.779324,1.095760,-0.359691,-0.000000,0.000000,0.000000,0.971746,1.545165e+01,RadioactiveDecay,eIoni
1046,290013,3,anti_nu_e,False,1,396.107330,393.276245,534.039307,0.000183,-8222.476562,...,-0.316265,0.444264,0.085812,-0.316265,0.444264,0.085812,0.552049,1.504398e+04,RadioactiveDecay,Transportation
1047,290013,2,Pb208[3475.078],False,1,396.107330,393.276245,534.039307,0.000183,396.107330,...,-0.463059,-1.540024,0.273879,-0.000000,-0.000000,0.000000,0.000007,0.000000e+00,RadioactiveDecay,RadioactiveDecay
1048,290013,6,gamma,False,2,396.107330,393.276245,534.039307,0.000183,460.731201,...,-0.527390,-0.391711,0.555856,0.000000,0.000000,-0.000000,0.860554,2.767462e+02,RadioactiveDecay,phot
1049,290013,21,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,-0.005523,-0.005359,0.000156,-0.000000,-0.000000,0.000000,0.000058,1.447787e-07,phot,msc
1050,290013,20,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,0.006145,-0.002717,-0.004100,-0.000000,0.000000,0.000000,0.000061,1.473075e-07,phot,msc
1051,290013,19,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,0.008027,0.014151,-0.023069,0.000000,0.000000,-0.000000,0.000779,1.385419e-05,phot,eIoni
1052,290013,18,gamma,False,6,460.731201,331.111450,540.388916,0.923309,460.740753,...,0.006884,0.002067,0.003525,0.000000,-0.000000,0.000000,0.008006,1.171419e-02,phot,phot
1053,290013,24,e-,False,18,460.740753,331.106476,540.393433,0.923348,460.740753,...,-0.022438,0.020111,-0.005542,-0.000000,0.000000,-0.000000,0.000918,1.704574e-05,phot,eIoni


### Need a bit of code to check and make sure that the parent particle of these positrons is Xe136

In [9]:
# select all positrons

# create tuple of event_id - mother_id to select from

positron_event_mothers = ((MC_df[MC_df.particle_name == 'e+']).event_id.to_numpy(), (MC_df[MC_df.particle_name == 'e+']).mother_id.to_numpy())
display(positron_event_mothers[0][10])
display(positron_event_mothers[1][10])
display(MC_df[MC_df.event_id == 290078])
display(MC_df[MC_df.particle_name == 'Xe131'])

290078

44

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
6379,290078,1,Tl208,True,0,397.517181,393.651123,533.796814,0.000000,397.517181,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,none,RadioactiveDecay
6380,290078,4,e-,False,1,397.517181,393.651123,533.796814,0.000370,396.172668,...,0.226810,-0.541064,-0.082542,0.000000,0.000000,-0.000000,2.713859e-01,3.407538e+00,RadioactiveDecay,eIoni
6381,290078,3,anti_nu_e,False,1,397.517181,393.651123,533.796814,0.000370,-906.634766,...,-0.108366,-0.753754,0.994305,-0.108366,-0.753754,0.994305,1.252410e+00,1.507243e+04,RadioactiveDecay,Transportation
6382,290078,2,Pb208[3475.078],False,1,397.517181,393.651123,533.796814,0.000370,397.517181,...,-0.118445,1.294818,-0.911763,-0.000000,0.000000,-0.000000,6.510090e-06,0.000000e+00,RadioactiveDecay,RadioactiveDecay
6383,290078,6,gamma,False,2,397.517181,393.651123,533.796814,0.000370,414.921906,...,0.272301,-0.010746,-0.051663,0.000000,-0.000000,-0.000000,2.773668e-01,1.772853e+01,RadioactiveDecay,phot
6384,290078,12,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,0.002271,-0.007032,0.002156,0.000000,-0.000000,-0.000000,5.797000e-05,1.447787e-07,phot,msc
6385,290078,11,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,-0.004965,-0.005903,0.000208,-0.000000,-0.000000,0.000000,5.826000e-05,1.450558e-07,phot,msc
6386,290078,10,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,-0.002859,0.000033,-0.005115,-0.000000,0.000000,-0.000000,3.360000e-05,1.221262e-07,phot,msc
6387,290078,9,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,0.018118,-0.018195,-0.009791,0.000000,-0.000000,-0.000000,7.383700e-04,1.259782e-05,phot,eIoni
6388,290078,8,gamma,False,6,414.921906,392.964264,530.494629,0.059506,414.936340,...,0.001473,0.002954,0.007294,0.000000,0.000000,0.000000,8.005710e-03,7.840825e-02,phot,phot


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3120,290038,28,Xe131,False,25,-329.410553,152.934982,779.154663,2.691185,-329.410675,...,-0.342947,-0.645131,-0.134031,-0.0,-0.0,-0.0,0.000002,0.000309,conv,NoProcess
14488,290178,66,Xe131,False,63,186.531647,109.520004,773.333679,1.423868,186.531677,...,0.052582,-0.006367,0.990931,0.0,-0.0,0.0,0.000004,0.000412,conv,NoProcess
17632,290218,43,Xe131,False,40,-86.615578,82.166649,588.105042,1.924311,-86.615837,...,-0.629226,0.011447,-0.349185,-0.0,0.0,-0.0,0.000002,0.000299,conv,NoProcess
22073,290273,32,Xe131,False,29,108.491875,381.571106,885.146973,1.517249,108.491722,...,-0.366685,-0.736688,0.442954,-0.0,-0.0,0.0,0.000004,0.000388,conv,NoProcess


In [62]:
MC_df.final_proc.unique()

array(['RadioactiveDecay', 'eIoni', 'phot', 'msc', 'Transportation',
       'NoProcess', 'conv', 'annihil', 'UserSpecialCut'], dtype=object)

In [63]:
MC_df.creator_proc.unique()

array(['none', 'RadioactiveDecay', 'eBrem', 'phot', 'compt', 'eIoni',
       'conv', 'annihil'], dtype=object)

In [10]:
MC_df[MC_df.creator_proc == 'conv']
# this line here has the sauce, it shows the events oyu care about

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
1085,290013,45,Cu65,False,42,402.410797,398.662048,547.204712,0.052081,402.410797,...,0.992594,-0.040111,0.144458,0.0,-0.0,0.0,8.333067e-06,1.445683e-06,conv,NoProcess
1086,290013,44,e+,False,42,402.410797,398.662048,547.204712,0.052081,402.561707,...,0.319938,0.809486,1.214907,0.0,-0.0,-0.0,1.068478e+00,9.174533e-01,conv,annihil
1128,290013,43,e-,False,42,402.410797,398.662048,547.204712,0.052081,402.434662,...,-0.253277,0.135666,0.852989,0.0,-0.0,0.0,5.240200e-01,3.302958e-01,conv,eIoni
2322,290031,40,Pb207,False,33,-839.102844,-34.995232,1319.603027,5.131673,-839.102844,...,-0.906009,0.287164,0.020794,-0.0,0.0,0.0,2.344284e-06,7.920021e-07,conv,NoProcess
2323,290031,39,e+,False,33,-839.102844,-34.995232,1319.603027,5.131673,-839.201111,...,-0.957210,-0.302034,0.610646,0.0,-0.0,-0.0,7.702054e-01,4.256005e-01,conv,annihil
2433,290031,38,e-,False,33,-839.102844,-34.995232,1319.603027,5.131673,-839.083252,...,0.068201,-0.577608,0.373589,-0.0,0.0,-0.0,3.486351e-01,1.569769e-01,conv,eIoni
2485,290032,50,Cu65,False,47,399.876465,387.161316,525.278625,0.052235,399.876465,...,0.106625,-0.306335,0.001459,0.0,-0.0,0.0,8.700226e-07,4.671279e-07,conv,NoProcess
2486,290032,49,e+,False,47,399.876465,387.161316,525.278625,0.052235,399.838470,...,0.425265,-1.011116,-1.014296,-0.0,0.0,-0.0,1.067963e+00,9.174178e-01,conv,annihil
2523,290032,48,e-,False,47,399.876465,387.161316,525.278625,0.052235,399.907715,...,0.352634,-0.785583,-0.264073,-0.0,-0.0,0.0,5.245423e-01,3.154818e-01,conv,eIoni
3120,290038,28,Xe131,False,25,-329.410553,152.934982,779.154663,2.691185,-329.410675,...,-0.342947,-0.645131,-0.134031,-0.0,-0.0,-0.0,2.263027e-06,3.086939e-04,conv,NoProcess


In [36]:
signal_df = pd.DataFrame(columns = MC_df.columns.values)

conv_slice = MC_df[MC_df.creator_proc == 'conv']
for e_id, df in conv_slice.groupby('event_id'):
    # check if conv selection has Xe isotope
    lst = df.particle_name.to_list()
    # truncate to only take 'Xe' as the input
    short_lst = []
    [short_lst.append(i[0:2]) for i in lst]
    if "Xe" in short_lst:
        display(df)
        print("Xe found in event {}".format(e_id))
        signal_df = pd.concat([signal_df, MC_df[MC_df.event_id == e_id]])

display(signal_df)


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3120,290038,28,Xe131,False,25,-329.410553,152.934982,779.154663,2.691185,-329.410675,...,-0.342947,-0.645131,-0.134031,-0.0,-0.0,-0.0,0.000002,0.000309,conv,NoProcess
3121,290038,27,e+,False,25,-329.410553,152.934982,779.154663,2.691185,-332.519196,...,-0.505731,0.022112,-0.069614,-0.0,-0.0,-0.0,0.211648,34.512207,conv,annihil
3157,290038,26,e-,False,25,-329.410553,152.934982,779.154663,2.691185,-245.353363,...,-1.512125,-0.167821,1.001654,0.0,-0.0,-0.0,1.380856,379.070221,conv,eIoni


Xe found in event 290038


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3618,290044,21,Xe129,False,18,-31.585411,362.857666,964.609985,2.032441,-31.585423,...,-0.027980,-0.129073,0.473258,-0.0,-0.0,0.0,0.000001,0.000204,conv,NoProcess
3619,290044,20,e+,False,18,-31.585411,362.857666,964.609985,2.032441,-29.040545,...,-0.357743,-0.177293,0.230968,0.0,0.0,0.0,0.177390,28.071030,conv,annihil
3665,290044,19,e-,False,18,-31.585411,362.857666,964.609985,2.032441,-16.297134,...,-1.444328,0.155733,1.156918,-0.0,-0.0,-0.0,1.415115,383.216431,conv,eIoni


Xe found in event 290044


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3901,290048,35,Xe129,False,32,294.954315,305.864624,308.7677,0.876499,294.954559,...,0.636000,-0.470344,-0.694365,0.0,-0.0,-0.0,0.000005,0.000437,conv,NoProcess
3902,290048,34,e+,False,32,294.954315,305.864624,308.7677,0.876499,258.061798,...,-0.597550,0.056914,-0.879747,0.0,-0.0,-0.0,0.670263,186.925034,conv,annihil
3937,290048,33,e-,False,32,294.954315,305.864624,308.7677,0.876499,213.382721,...,-1.054603,-0.504079,-0.653273,-0.0,0.0,-0.0,0.922239,288.516418,conv,eIoni


...

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
26804,290337,35,Xe129,False,32,-188.727936,-232.105347,636.679871,2.884029,-188.728149,...,-0.531832,-0.255082,-0.175839,-0.0,-0.0,-0.0,0.000002,0.000256,conv,NoProcess
26805,290337,34,e+,False,32,-188.727936,-232.105347,636.679871,2.884029,-247.317276,...,-0.947371,-1.613477,0.156932,-0.0,0.0,-0.0,1.434911,424.788208,conv,annihil
26854,290337,33,e-,False,32,-188.727936,-232.105347,636.679871,2.884029,-192.305084,...,-0.289582,-0.033484,0.317674,-0.0,-0.0,0.0,0.157593,16.216393,conv,eIoni


Xe found in event 290337


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
28207,290353,42,Xe129,False,39,202.613174,227.756912,632.450989,0.921263,202.613022,...,-0.341951,-0.435267,-0.012501,-0.0,-0.0,-0.0,0.000001,0.000230,conv,NoProcess
28208,290353,41,e+,False,39,202.613174,227.756912,632.450989,0.921263,202.296387,...,0.043773,-0.268706,0.184241,0.0,-0.0,0.0,0.096606,9.072005,conv,annihil
28266,290353,40,e-,False,39,202.613174,227.756912,632.450989,0.921263,183.060043,...,-1.532714,-0.906207,0.772075,0.0,0.0,0.0,1.495900,441.856415,conv,eIoni


Xe found in event 290353


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3096,290038,1,Tl208,True,0,399.094788,396.975037,532.901978,0.000000,399.094788,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,none,RadioactiveDecay
3097,290038,4,e-,False,1,399.094788,396.975037,532.901978,0.000011,399.597656,...,0.317499,-0.759870,-1.221578,-0.00000,-0.000000,0.000000,1.048354,2.959284,RadioactiveDecay,eIoni
3098,290038,3,anti_nu_e,False,1,399.094788,396.975037,532.901978,0.000011,-4433.631836,...,-0.149950,-0.200074,-0.404385,-0.14995,-0.200074,-0.404385,0.475439,15322.882812,RadioactiveDecay,Transportation
3099,290038,2,Pb208[3475.078],False,1,399.094788,396.975037,532.901978,0.000011,399.094788,...,-0.167549,0.959945,1.625964,-0.00000,0.000000,0.000000,0.000009,0.000000,RadioactiveDecay,RadioactiveDecay
3100,290038,6,gamma,False,2,399.094788,396.975037,532.901978,0.000011,433.833130,...,0.663099,0.249662,-0.488388,0.00000,-0.000000,-0.000000,0.860554,64.621887,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28285,290353,108,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.107101,...,0.012740,-0.000458,-0.003118,0.00000,-0.000000,-0.000000,0.000169,0.001002,phot,eIoni
28286,290353,107,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.125565,...,0.092590,-0.026467,-0.076831,-0.00000,-0.000000,0.000000,0.014640,0.404148,phot,eIoni
28287,290353,101,e-,False,40,183.980576,147.294357,703.405212,1.336698,183.735596,...,-0.114430,-0.104642,-0.072460,0.00000,-0.000000,0.000000,0.027902,1.239423,eIoni,eIoni
28288,290353,100,e-,False,40,199.922455,176.166901,671.844849,1.157253,199.694321,...,-0.134355,-0.004156,0.091696,-0.00000,-0.000000,-0.000000,0.025281,1.114063,eIoni,eIoni


In [38]:
# save signal_df as events
signal_df.to_hdf('Tl_signal.h5', key = 'MC')

/home/e78368jw/anaconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['event_id', 'particle_id', 'particle_name', 'primary', 'mother_id',
       'initial_volume', 'final_volume', 'creator_proc', 'final_proc'],
      dtype='object')]

  pytables.to_hdf(


In [40]:
load_test = pd.read_hdf('Tl_signal.h5', key = 'MC')

In [41]:
display(load_test)

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3096,290038,1,Tl208,True,0,399.094788,396.975037,532.901978,0.000000,399.094788,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,none,RadioactiveDecay
3097,290038,4,e-,False,1,399.094788,396.975037,532.901978,0.000011,399.597656,...,0.317499,-0.759870,-1.221578,-0.00000,-0.000000,0.000000,1.048354,2.959284,RadioactiveDecay,eIoni
3098,290038,3,anti_nu_e,False,1,399.094788,396.975037,532.901978,0.000011,-4433.631836,...,-0.149950,-0.200074,-0.404385,-0.14995,-0.200074,-0.404385,0.475439,15322.882812,RadioactiveDecay,Transportation
3099,290038,2,Pb208[3475.078],False,1,399.094788,396.975037,532.901978,0.000011,399.094788,...,-0.167549,0.959945,1.625964,-0.00000,0.000000,0.000000,0.000009,0.000000,RadioactiveDecay,RadioactiveDecay
3100,290038,6,gamma,False,2,399.094788,396.975037,532.901978,0.000011,433.833130,...,0.663099,0.249662,-0.488388,0.00000,-0.000000,-0.000000,0.860554,64.621887,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28285,290353,108,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.107101,...,0.012740,-0.000458,-0.003118,0.00000,-0.000000,-0.000000,0.000169,0.001002,phot,eIoni
28286,290353,107,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.125565,...,0.092590,-0.026467,-0.076831,-0.00000,-0.000000,0.000000,0.014640,0.404148,phot,eIoni
28287,290353,101,e-,False,40,183.980576,147.294357,703.405212,1.336698,183.735596,...,-0.114430,-0.104642,-0.072460,0.00000,-0.000000,0.000000,0.027902,1.239423,eIoni,eIoni
28288,290353,100,e-,False,40,199.922455,176.166901,671.844849,1.157253,199.694321,...,-0.134355,-0.004156,0.091696,-0.00000,-0.000000,-0.000000,0.025281,1.114063,eIoni,eIoni


In [3]:
# develop a function that can load in files, collect them like this, and add to a large dataframe.
def collect_signal_df(data_path, save_path = "", verbose = True):
    """
    Function that collects Tl208 signal events (identified with Xe ions created in 'conv' processes)
    """

     # collect all filenames
    try:
        file_names = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and f.endswith('.h5')]
    except:
        print("File path incorrect, please state the correct file path\n(but not any particular folder!)")

    # counter for creating first array
    i = 0


    for file in tqdm(file_names):
        file_path = data_path + file

        # load in data
        MC_df = pd.read_hdf(file_path, 'MC/particles')
        
        # set first dataframe
        if (i == 0):
            MC_signal_df = pd.DataFrame(columns = MC_df.columns.values)
            i = 1
        
        # collect conv slice and iterate over, scanning for Xe*** events
        conv_slice = MC_df[MC_df.creator_proc == 'conv']
        for e_id, df in conv_slice.groupby('event_id'):
            # check if conv selection has Xe isotope
            lst = df.particle_name.to_list()
            # truncate to only take 'Xe' as the input
            short_lst = []
            [short_lst.append(i[0:2]) for i in lst]
            if "Xe" in short_lst:
                if (verbose == True):
                    display(df)
                    print("Xe found in event {}".format(e_id))
                MC_signal_df = pd.concat([MC_signal_df, MC_df[MC_df.event_id == e_id]])

    # save if desired
    if save_path != "":
        MC_signal_df.to_hdf(str(save_path) + 'Tl_signal.h5', key = 'MC')

    # return dataframe    
    return MC_signal_df

def collect_soph_signal_df(data_path, event_ids, save_path = "", MC_check = True):
    '''

    Function that collects all sophronia signal events and stores them in a file

    This will produce a dataframe with all the 'RECO/Events' data
    for the relevant events.

    data_path -> path to the sophronia files
    event_ids -> array including all the event ids
    save_path -> path under which to save the data
    MC_check  -> check if the event ids are from MC, if so, they are doubled
    '''

    if MC_check == True:
        event_ids = event_ids * 2
         # collect all filenames
    try:
        file_names = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and f.endswith('.h5')]
    except:
        print("File path incorrect, please state the correct file path\n(but not any particular folder!)")

    # counter for creating first array
    i = 0


    #

    for file in tqdm(file_names):
        file_path = data_path + file

        # load in data
        soph_df = dstio.load_dst(file_path, 'RECO', 'Events')
        
        # set first dataframe
        if (i == 0):
            soph_signal_df = pd.DataFrame(columns = soph_df.columns.values)
            i = 1

        
        # collect unique event ids from file, and then find how many are signals 
        ids = np.array(soph_df.event.unique())
        available_evts = np.intersect1d(ids, event_ids)

        # collect the signal events from the files
        for i in available_evts:
            soph_signal_df = pd.concat([soph_signal_df, soph_df[soph_df.event == i]])
    
        # save if desired
    if save_path != "":
        soph_signal_df.to_hdf(str(save_path) + 'Tl_signal_soph.h5', key = 'MC')

    return soph_signal_df


def collect_signal_true_hits(data_path, event_ids, save_path = ""):
    '''

    Function that collects all sophronia signal events and stores them in a file

    This will produce a dataframe with all the 'RECO/Events' data
    for the relevant events.

    data_path -> path to the sophronia files
    event_ids -> array including all the event ids
    save_path -> path under which to save the data
    MC_check  -> check if the event ids are from MC, if so, they are doubled
    '''

    try:
        file_names = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and f.endswith('.h5')]
    except:
        print("File path incorrect, please state the correct file path\n(but not any particular folder!)")

    # counter for creating first array
    i = 0


    #

    for file in tqdm(file_names):
        file_path = data_path + file

        # load in data
        true_info = mcio.load_mchits_df(file_path).reset_index()
        
        # set first dataframe
        if (i == 0):
            true_info_df = pd.DataFrame(columns = true_info.columns.values)
            i = 1

        
        # collect unique event ids from file, and then find how many are signals 
        ids = np.array(true_info.event_id.unique())
        available_evts = np.intersect1d(ids, event_ids)

        # collect the signal events from the files
        for r in available_evts:
            true_info_df = pd.concat([true_info_df, true_info[true_info.event_id == r]])
    
        # save if desired
    if save_path != "":
        true_info_df.to_hdf(str(save_path) + 'Tl_signal_true_info.h5', key = 'MC')

    return true_info_df

        

In [21]:
data = collect_signal_df(r'/home/e78368jw/Documents/NEXT_CODE/Tl_studies/LPR/208Tl/PORT_1a/prod/sophronia/sophronia/', r'/home/e78368jw/Documents/NEXT_CODE/Tl_studies/LPR/208Tl/PORT_1a/prod/sophronia/', verbose = True)


  0%|          | 0/300 [00:00<?, ?it/s]

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
936,140010,17,Xe129,False,14,348.010773,86.433571,313.230255,1.286397,348.010742,...,-0.074046,-0.418765,-0.036050,-0.0,-0.0,-0.0,7.586495e-07,0.000177,conv,NoProcess
937,140010,16,e+,False,14,348.010773,86.433571,313.230255,1.286397,336.748810,...,-0.331915,-1.357668,-0.724240,-0.0,-0.0,-0.0,1.144016e+00,340.518494,conv,annihil
965,140010,15,e-,False,14,348.010773,86.433571,313.230255,1.286397,334.428619,...,0.079886,-0.346918,-0.729907,-0.0,0.0,-0.0,4.484898e-01,92.513046,conv,eIoni


Xe found in event 140010


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3719,140047,58,Xe131,False,55,-304.90097,-335.727875,410.826477,3.401662,-304.900970,...,-0.044403,-0.544059,0.075794,-0.0,-0.0,0.0,0.000001,0.000229,conv,NoProcess
3720,140047,57,e+,False,55,-304.90097,-335.727875,410.826477,3.401662,-327.399628,...,-1.518872,-0.732606,-0.346366,0.0,-0.0,0.0,1.284766,47.402359,conv,annihil
3743,140047,56,e-,False,55,-304.90097,-335.727875,410.826477,3.401662,-309.964325,...,-0.238171,-0.592218,-0.042013,0.0,0.0,0.0,0.307739,48.067650,conv,eIoni


Xe found in event 140047


...

Xe found in event 1150334


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
26030,1150336,60,Xe132,False,57,287.928223,314.638,421.554077,0.59633,287.928192,...,-0.076115,-0.174124,-0.462129,-0.0,-0.0,-0.0,0.000001,0.000208,conv,NoProcess
26031,1150336,59,e+,False,57,287.928223,314.638,421.554077,0.59633,291.432434,...,-0.363799,-0.284792,-0.823662,-0.0,-0.0,-0.0,0.562777,141.088409,conv,annihil
26066,1150336,58,e-,False,57,287.928223,314.638,421.554077,0.59633,262.357117,...,-1.207883,-0.720990,-0.365943,0.0,0.0,0.0,1.029729,286.394531,conv,eIoni


Xe found in event 1150336


  4%|▎         | 11/300 [00:16<07:26,  1.54s/it]


KeyboardInterrupt: 

In [30]:
print(q)

[140010 140047 140052 ... 200280 200284 200311]


In [38]:
true_info = collect_signal_true_hits(r'/home/e78368jw/Documents/NEXT_CODE/Tl_studies/LPR/208Tl/PORT_1a/prod/sophronia/sophronia/', q, save_path = "./")

100%|██████████| 300/300 [19:29<00:00,  3.90s/it]
/home/e78368jw/anaconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['event_id', 'particle_id', 'hit_id', 'label'], dtype='object')]

  pytables.to_hdf(


In [8]:
true_info = pd.read_hdf('Tl_signal_true_info.h5')

In [39]:
#true_info = mcio.load_mchits_df(r'/home/e78368jw/Documents/NEXT_CODE/Tl_studies/LPR/208Tl/PORT_1a/prod/sophronia/sophronia/sophronia_1_208Tl.h5').reset_index()
display(true_info)

,event_id,particle_id,hit_id,x,y,z,time,energy,label
3569,140010,17,0,348.010742,86.433395,313.230225,1.452806,7.586495e-07,ACTIVE
3570,140010,16,0,347.988129,85.574280,312.733521,1.289878,4.572421e-03,ACTIVE
3571,140010,16,1,348.067047,84.715431,312.227966,1.293386,1.695531e-03,ACTIVE
3572,140010,16,2,348.156677,83.887016,311.701508,1.296844,1.123778e-03,ACTIVE
3573,140010,16,3,348.156158,82.904945,311.521088,1.300347,1.956159e-03,ACTIVE
...,...,...,...,...,...,...,...,...,...
117993,200311,54,12,-150.695969,-58.671600,154.616302,3.017361,1.371744e-03,ACTIVE
117994,200311,54,13,-150.690796,-58.666924,154.624374,3.017602,3.005840e-03,ACTIVE
117995,200311,54,14,-150.690460,-58.664375,154.627060,3.017724,6.758421e-04,ACTIVE
117996,200311,54,15,-150.691635,-58.662224,154.626572,3.017819,7.255733e-04,ACTIVE


In [15]:
display(data[data.creator_proc == 'conv'].event_id.nunique())

6175

In [16]:
display(data.event_id.nunique())

6175

In [22]:
signal_ids = np.array(data.event_id.unique())
print(signal_ids)
np.save('signal_ids.npy', signal_ids)

[140010 140047 140052 ... 200280 200284 200311]


In [4]:
q = np.load('signal_ids.npy', allow_pickle=True)
print(q)

[140010 140047 140052 ... 200280 200284 200311]


In [33]:
ids = np.array(MC_df.event_id.unique())


signal_in_data = np.intersect1d(ids, q)

print(signal_in_data)


[290038 290044 290048 290055 290078 290099 290106 290108 290114 290155
 290159 290178 290218 290253 290261 290273 290278 290304 290310 290315
 290337 290353]


In [47]:
print(q[580:1000])

[280275 280278 280300 280302 280312 280331 280334 280341 290038 290044
 290048 290055 290078 290099 290106 290108 290114 290155 290159 290178
 290218 290253 290261 290273 290278 290304 290310 290315 290337 290353
 300037 300091 300095 300139 300182 300194 300199 300207 300230 300242
 300243 300253 300281 300298 300309 300331 310019 310021 310022 310024
 310026 310044 310064 310080 310088 310100 310103 310128 310131 310154
 310156 310162 310164 310193 310207 310217 310223 310258 310259 310265
 310274 310275 310306 310321 310326 320016 320018 320021 320031 320035
 320046 320053 320057 320077 320084 320097 320118 320124 320127 320163
 320184 320189 320211 320215 320247 320256 320265 320271 320274 320304
 330007 330014 330026 330038 330046 330054 330081 330089 330129 330140
 330155 330178 330189 330191 330205 330207 330217 330231 330267 330285
 330287 330291 330295 330296 330312 330316 330320 330326 330336 340003
 340019 340039 340044 340054 340069 340073 340075 340090 340124 340127
 34014

In [56]:
soph_data = collect_soph_signal_df(r'/home/e78368jw/Documents/NEXT_CODE/Tl_studies/LPR/208Tl/PORT_1a/prod/sophronia/sophronia/', q, save_path = "./", MC_check = True)

100%|██████████| 300/300 [27:52<00:00,  5.57s/it]
/home/e78368jw/anaconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block1_values] [items->Index(['event', 'npeak', 'nsipm', 'track_id'], dtype='object')]

  pytables.to_hdf(


In [57]:
display(soph_data)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
4602,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,23.325,0.0,0.0,228.426971,6.782609,913.878089,-1.0,0.002869,-1,-1.0
4603,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,23.325,0.0,0.0,228.426971,7.014493,945.121843,-1.0,0.003000,-1,-1.0
4604,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,23.325,0.0,0.0,229.452454,8.811594,597.846741,-1.0,0.001877,-1,-1.0
4605,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,38.875,0.0,0.0,229.452454,6.666667,452.318298,-1.0,0.001420,-1,-1.0
4606,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,23.325,0.0,0.0,229.452454,8.637681,586.047180,-1.0,0.001860,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164089,400622,4.006231e+11,0,-171.015439,-59.059529,1,-116.625,-54.425,0.0,0.0,246.149994,8.521739,340.697388,-1.0,0.001034,-1,-1.0
164090,400622,4.006231e+11,0,-171.015439,-59.059529,1,-101.075,-54.425,0.0,0.0,246.149994,5.391304,215.543259,-1.0,0.000654,-1,-1.0
164091,400622,4.006231e+11,0,-171.015439,-59.059529,1,-132.175,-69.975,0.0,0.0,247.165451,6.144928,752.258115,-1.0,0.002307,-1,-1.0
164092,400622,4.006231e+11,0,-171.015439,-59.059529,1,-132.175,-54.425,0.0,0.0,247.165451,6.608696,809.032262,-1.0,0.002477,-1,-1.0


In [58]:
print(soph_data.event.nunique())

6175


### importing all the relevant functions here

In [12]:
import sys,os,os.path
#sys.path.append("../../")   # cite IC from parent directory
sys.path.append("/gluster/data/next/software/IC_satkill/")
#sys.path.append("/gluster/data/next/software/IC_sophronia/")
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/gluster/data/next/software/IC_satkill/'


#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

import time

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clrs
from matplotlib.lines import Line2D


import IC.invisible_cities.core.core_functions                   as     coref
import IC.invisible_cities.io.dst_io                           as     dstio

from IC.invisible_cities.cities                 import beersheba as beerfun

from IC.invisible_cities.evm.event_model                          import HitCollection

from IC.invisible_cities.database.load_db       import DataSiPM

from IC.invisible_cities.evm.event_model        import Cluster, Hit
from IC.invisible_cities.types.ic_types         import xy
from IC.invisible_cities.reco.paolina_functions import voxelize_hits, drop_end_point_voxels, make_track_graphs, get_track_energy

from IC.invisible_cities.evm.event_model        import HitEnergy
from IC.invisible_cities.cities.beersheba          import DeconvolutionMode
from IC.invisible_cities.cities.beersheba          import CutType

from IC.invisible_cities.reco import hits_functions as hif

from IC.invisible_cities.reco.deconv_functions import deconvolve
from IC.invisible_cities.reco.deconv_functions import deconvolution_input
from IC.invisible_cities.reco.deconv_functions import InterpolationMethod

import IC.invisible_cities.io.mcinfo_io as mcio

from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.colors import Normalize



def threshold_hits(threshold_charge, same_peak, hitc):
    """
    Applies a threshold to hits and redistributes the charge/energy.

    Parameters
    ----------
    threshold_charge : float
        minimum pes of a hit
    same_peak        : bool
        whether to reassign NN hits' energy only to the hits from the same peak

    Returns
    ----------
    A function that takes HitCollection as input and returns another object with
    only non NN hits of charge above threshold_charge.
    The energy of NN hits is redistributed among neighbors.
    """

    t = hitc.time
    thr_hits = hif.threshold_hits(hitc.hits, threshold_charge     )
    mrg_hits = hif.merge_NN_hits ( thr_hits, same_peak = same_peak)

    cor_hits = []
    for hit in mrg_hits:
        cluster = Cluster(hit.Q, xy(hit.X, hit.Y), hit.var, hit.nsipm)
        xypos   = xy(hit.Xpeak, hit.Ypeak)
        hit     = Hit(hit.npeak, cluster, hit.Z, hit.E, xypos, hit.Ec)
        cor_hits.append(hit)

    new_hitc      = HitCollection(hitc.event, t)
    new_hitc.hits = cor_hits
    return new_hitc





def hits_from_df (dst, skip_NN = False):
    """
    Function that transforms pandas DataFrame dst to HitCollection
    ------
    Parameters
    ------
    dst : pd.DataFrame
        DataFrame with obligatory columns :
                event, npeak, X, Y, Z,  Q, E
        If time, nsipm, Xrms, Yrms, Qc, Ec, track_id are not
        inside dst the default value is set to -1
        If Xpeak, Ypeak not in dst the default value is -1000
    ------
    Returns
    ------
    Dictionary {event_number : HitCollection}
    from here
    https://github.com/next-exp/IC/blob/v2-development/invisible_cities/io/hits_io.py#L16
    """
    all_events = {}
    times = getattr(dst, 'time', [-1]*len(dst))
    for (event, time) , df in dst.groupby(['event', times]):
        #pandas is not consistent with numpy dtypes so we have to change it by hand
        event = np.int32(event)
        hits  = []
        for i, row in df.iterrows():
            Q = getattr(row,'Q', row.E)
            if skip_NN and Q == NN:
                continue
            if hasattr(row, 'Xrms'):
                Xrms  = row.Xrms
                Xrms2 = Xrms**2
            else:
                Xrms = Xrms2 = -1
            if hasattr(row, 'Yrms'):
                Yrms  = row.Yrms
                Yrms2 = Yrms**2
            else:
                Yrms = Yrms2 = -1
            nsipm   = getattr(row, 'nsipm'   , -1   )     # for backwards compatibility
            Qc      = getattr(row, 'Qc'      , -1   )     # for backwards compatibility
            Xpeak   = getattr(row, 'Xpeak'   , -1000)     # for backwards compatibility
            Ypeak   = getattr(row, 'Ypeak'   , -1000)     # for backwards compatibility
            Ec      = getattr(row, 'Ec'      , -1   )     # for backwards compatibility
            trackID = getattr(row, 'track_id', -1   )     # for backwards compatibility
            Ep      = getattr(row, "Ep"      , -1   )     # for backwards compatibility

            hit = Hit(row.npeak            ,
                      Cluster(Q               ,
                              xy(row.X, row.Y),
                              xy(Xrms2, Yrms2),
                              nsipm = nsipm   ,
                              z     = row.Z   ,
                              E     = row.E   ,
                              Qc    = Qc      ),
                      row.Z                ,
                      row.E                ,
                      xy(Xpeak, Ypeak)     ,
                      s2_energy_c = Ec     ,
                      track_id    = trackID,
                      Ep          = Ep     )

            hits.append(hit)

        if len(hits):
            all_events[event] = HitCollection(event, time, hits=hits)

    return all_events

def hitc_to_df_(hitc):
    columns = defaultdict(list)
    for hit in hitc.hits:
        columns["event"   ].append(hitc.event)
        columns["time"    ].append(hitc.time)
        columns["npeak"   ].append(hit .npeak)
        columns["Xpeak"   ].append(hit .Xpeak)
        columns["Ypeak"   ].append(hit .Ypeak)
        columns["nsipm"   ].append(hit .nsipm)
        columns["X"       ].append(hit .X)
        columns["Y"       ].append(hit .Y)
        columns["Xrms"    ].append(hit .Xrms)
        columns["Yrms"    ].append(hit .Yrms)
        columns["Z"       ].append(hit .Z)
        columns["Q"       ].append(hit .Q)
        columns["E"       ].append(hit .E)
        columns["Qc"      ].append(hit .Qc)
        columns["Ec"      ].append(hit .Ec)
        columns["track_id"].append(hit .track_id)
        columns["Ep"      ].append(hit .Ep)
    return pd.DataFrame(columns)






def soph_to_lowTh(df, threshold = 5):
    '''
    Converts sophronia 'RECO/Events' to lowTh events via a rather convoluted process
    Made by me (John Waiton), so dont treat it like a normal function from IC!
    ------
    Parameters
    ------
    df : pd.DataFrame
        DataFrame with obligatory columns :
                event, npeak, X, Y, Z,  Q, E
    threshold: int
        value at which the threshold is set.
    ------
    Returns
    ------
    Dictionary {event_number : HitCollection}
    from here
    '''

    # safety check, to ensure you don't accidentally make a repeating dataframe
    


    # new parameters for threshold, this is silly but I'm copying previous convention
    pes = 1
    threshold = threshold * pes
    same_peak = True

    # convert sophronia RECO/Events to hit collection
    soph_hitc = hits_from_df(df)

    # collect the keys as the event numbers
    soph_hitc_list = list(soph_hitc.keys())

    print("Processing data...")
    # loop over all of these events
    j = 0
    for i in soph_hitc_list:
        j += 1

        if (len(soph_hitc_list)%j == 50): 
            print("{}/{}".format(j, len(soph_hitc_list)))
        # choose i'th event
        soph_hit_event = soph_hitc.get(i)

        # Apply threshold calculation
        soph_hitc_lowTh = threshold_hits(threshold, same_peak, soph_hit_event)

        # convert back to pandas dataframe using hitc_to_df
        soph_hdst_lowTh = hitc_to_df_(soph_hitc_lowTh)

        # check if pandas dataframe with all the events exists yet
        if 'full_soph_df' in locals() and isinstance(full_soph_df, pd.DataFrame):
            full_soph_df = pd.concat([full_soph_df, soph_hdst_lowTh])
        else:
            full_soph_df = soph_hdst_lowTh.copy(deep = True)
    
    return full_soph_df

def collect_min_max_bins(hits):
    '''
    returns all the min, max and mid values you'd need
    as well as the bins
    '''
    x_range = (hits.X.max()-hits.X.min())/2.
    y_range = (hits.Y.max()-hits.Y.min())/2.
    z_range = (hits.Z.max()-hits.Z.min())/2.
    mid_x   = (hits.X.max()+hits.X.min())/2.
    mid_y   = (hits.Y.max()+hits.Y.min())/2.
    mid_z   = (hits.Z.max()+hits.Z.min())/2.
    min_x = hits.X.min()
    min_y = hits.Y.min()
    min_z = hits.Z.min()

    max_x = hits.X.max()
    max_y = hits.Y.max()
    max_z = hits.Z.max()
    #print("X maximum and minimum")
    #print(max_x, min_x)
    #print("")

    #print("Y maximum and minimum")
    #print(max_y, min_y)
    #print("")

    #print("Z maximum and minimum")
    #print(max_z, min_z)

    xbins = int(hits.X.max()-hits.X.min())
    ybins = int(hits.Y.max()-hits.Y.min())
    zbins = int((hits.Z.max()-hits.Z.min())/2.)
    
    array = [x_range, y_range, z_range, mid_x, mid_y, mid_z, min_x, min_y, min_z, max_x, max_y, max_z, xbins, ybins, zbins]
    
    return (array)



def count_tracks_mc(hits_deco):
   
    # stuff needed for paolina track counting
    energy_threshold = 10
    min_voxels = 3
    
    base_vsize = 12 #mm
    the_hits = []

    xs = hits_deco.x
    ys = hits_deco.y
    zs = hits_deco.z
    es = hits_deco.energy

    for x, y, z, e in zip(xs, ys, zs, es):
        if np.isnan(e): continue
        h = Hit(0, Cluster(0, xy(x,y), xy(0,0), 0), z, e*1000, xy(0,0))
        the_hits.append(h)

    voxels = voxelize_hits(the_hits,
                           np.array([base_vsize, base_vsize, base_vsize]), False)

    (mod_voxels, dropped_voxels) = drop_end_point_voxels(voxels, energy_threshold, min_voxels)

    
    tracks = make_track_graphs(mod_voxels)
    tracks = sorted(tracks, key=get_track_energy, reverse = True)
    
    track_no = 0
    for c, t in enumerate(tracks, 0):
        track_no += 1
    
    return track_no    

def return_id(number):
    return str(df_ps[df_ps.particle_id == number].particle_name.values).strip("'[]'")


def count_tracks(hits_deco):
    
    
    
    # stuff needed for paolina track counting
    energy_threshold = 10
    min_voxels = 3
    
    base_vsize = 12 #mm
    the_hits = []

    xs = hits_deco.X
    ys = hits_deco.Y
    zs = hits_deco.Z
    es = hits_deco.E

    for x, y, z, e in zip(xs, ys, zs, es):
        if np.isnan(e): continue
        h = Hit(0, Cluster(0, xy(x,y), xy(0,0), 0), z, e*1000, xy(0,0))
        the_hits.append(h)

    voxels = voxelize_hits(the_hits,
                           np.array([base_vsize, base_vsize, base_vsize]), False)

    (mod_voxels, dropped_voxels) = drop_end_point_voxels(voxels, energy_threshold, min_voxels)

    
    tracks = make_track_graphs(mod_voxels)
    tracks = sorted(tracks, key=get_track_energy, reverse = True)
    
    track_no = 0
    for c, t in enumerate(tracks, 0):
        track_no += 1
    
    return track_no

In [5]:
soph_data = pd.read_hdf('Tl_signal_soph.h5')
display(soph_data)
print(soph_data.event.nunique())

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
4602,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,23.325,0.0,0.0,228.426971,6.782609,913.878089,-1.0,0.002869,-1,-1.0
4603,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,23.325,0.0,0.0,228.426971,7.014493,945.121843,-1.0,0.003000,-1,-1.0
4604,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,23.325,0.0,0.0,229.452454,8.811594,597.846741,-1.0,0.001877,-1,-1.0
4605,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,38.875,0.0,0.0,229.452454,6.666667,452.318298,-1.0,0.001420,-1,-1.0
4606,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,23.325,0.0,0.0,229.452454,8.637681,586.047180,-1.0,0.001860,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164089,400622,4.006231e+11,0,-171.015439,-59.059529,1,-116.625,-54.425,0.0,0.0,246.149994,8.521739,340.697388,-1.0,0.001034,-1,-1.0
164090,400622,4.006231e+11,0,-171.015439,-59.059529,1,-101.075,-54.425,0.0,0.0,246.149994,5.391304,215.543259,-1.0,0.000654,-1,-1.0
164091,400622,4.006231e+11,0,-171.015439,-59.059529,1,-132.175,-69.975,0.0,0.0,247.165451,6.144928,752.258115,-1.0,0.002307,-1,-1.0
164092,400622,4.006231e+11,0,-171.015439,-59.059529,1,-132.175,-54.425,0.0,0.0,247.165451,6.608696,809.032262,-1.0,0.002477,-1,-1.0


6175


In [16]:
MC_soph_data = pd.read_hdf('Tl_signal.h5')
display(MC_soph_data)
print(MC_soph_data.event_id.nunique())
print(MC_soph_data[MC_soph_data.creator_proc == 'conv'].event_id.nunique())

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
923,140010,1,Tl208,True,0,396.073700,399.411133,532.882263,0.000000,396.073700,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,none,RadioactiveDecay
924,140010,4,e-,False,1,396.073700,399.411133,532.882263,0.000932,395.585052,...,-0.212988,0.805301,-0.743162,-0.000000,-0.000000,-0.000000,0.716716,2.029165,RadioactiveDecay,eIoni
925,140010,3,anti_nu_e,False,1,396.073700,399.411133,532.882263,0.000932,12500.000000,...,0.926451,0.288135,-0.484445,0.926451,0.288135,-0.484445,1.084445,14168.083984,RadioactiveDecay,Transportation
926,140010,2,Pb208[3197.711],False,1,396.073700,399.411133,532.882263,0.000932,396.073700,...,-0.713464,-1.093436,1.227607,-0.000000,-0.000000,0.000000,0.000008,0.000000,RadioactiveDecay,RadioactiveDecay
927,140010,6,gamma,False,2,396.073700,399.411133,532.882263,0.000932,398.100952,...,0.084283,-0.132974,0.561536,0.000000,-0.000000,0.000000,0.583188,14.027398,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24877,200311,64,e-,False,55,-169.080582,-34.253830,126.737892,3.155834,-169.080154,...,0.014782,0.011458,-0.008618,0.000000,0.000000,-0.000000,0.000415,0.002223,phot,eIoni
24878,200311,63,e-,False,55,-169.080582,-34.253830,126.737892,3.155834,-169.079819,...,0.004159,0.006084,0.058639,0.000000,0.000000,-0.000000,0.003406,0.027385,phot,eIoni
24879,200311,62,e-,False,55,-169.080582,-34.253830,126.737892,3.155834,-169.014648,...,-0.013530,0.121167,-0.042738,0.000000,-0.000000,0.000000,0.016079,0.538994,phot,eIoni
24880,200311,54,e-,False,27,-150.683487,-58.778706,154.704117,3.006442,-150.692184,...,0.141591,0.126753,-0.056101,-0.000000,-0.000000,-0.000000,0.037072,2.001722,eIoni,eIoni


6175
6175


In [14]:
q = np.load('signal_ids.npy', allow_pickle=True)
print(q)

[140010 140047 140052 ... 200280 200284 200311]


In [7]:
hdst = soph_data[soph_data.event == 280020]
display(hdst)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
4602,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,23.325,0.0,0.0,228.426971,6.782609,913.878089,-1.0,0.002869,-1,-1.0
4603,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,23.325,0.0,0.0,228.426971,7.014493,945.121843,-1.0,0.003000,-1,-1.0
4604,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,23.325,0.0,0.0,229.452454,8.811594,597.846741,-1.0,0.001877,-1,-1.0
4605,280020,2.800201e+11,0,332.154563,47.307991,1,318.775,38.875,0.0,0.0,229.452454,6.666667,452.318298,-1.0,0.001420,-1,-1.0
4606,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,23.325,0.0,0.0,229.452454,8.637681,586.047180,-1.0,0.001860,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5384,280020,2.800201e+11,0,332.154563,47.307991,1,349.875,85.525,0.0,0.0,318.592377,5.855072,223.330063,-1.0,0.000731,-1,-1.0
5385,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,54.425,0.0,0.0,319.649933,8.347826,544.716412,-1.0,0.001758,-1,-1.0
5386,280020,2.800201e+11,0,332.154563,47.307991,1,334.325,69.975,0.0,0.0,319.649933,9.391304,612.805931,-1.0,0.001983,-1,-1.0
5387,280020,2.800201e+11,0,332.154563,47.307991,1,349.875,69.975,0.0,0.0,319.649933,10.086957,658.199008,-1.0,0.002159,-1,-1.0


In [6]:
# set q_thr value here
thresh = 4

# Decides if plots are limited by the MC true info
lim_toggle = True


# limits
limit_spacing = 50 # put in the number required division-wise to produce a percentage
                    # eg: 10% of min and max means /10, 50% means /2


# process
#hdst = soph_to_lowTh(soph_data, threshold = thresh)

In [14]:
# Input file and some event IDs to look at.
dist = 12
typefile = 'esmeralda' #'esmeralda'
#file = f'/Users/halmamol/NEXT/files/NEXT100/separated/esmeralda/esmeralda_10_sep.h5'
#file = f'../../../Tl_studies/LPR/208Tl/PORT_2a/prod/esmeralda/esmeralda_180_208Tl.h5'
#file = f'/Users/halmamol/NEXT/files/FLEX/ELdistMask/Xe/bb0nu/{typefile}/NextFlex.bb0nu.FLEXELdist{dist}mask.0.detsim.h5'
evts = [90] #15, 30, 90
pitch = 15.55
detector_db     = '/home/e78368jw/Documents/NEXT_CODE/IC/invisible_cities/database/localdb.NEXT100DB.sqlite3'
run_number      = -1

datatype = 'MC'


# Parameters involved in beersheba city and the deconvolution.

'''
## FROM OLD TESTING
deconv_params = dict(
  q_cut           = 5, #5 for 6mm  - 5 for 6mm
  drop_dist       = [16, 16],
  #psf_fname       = f'/Users/halmamol/NEXT/PSF_FLEX100_mask{dist}_lowQE.h5',     
  psf_fname       = f'../data/data/LightTables/NEXT100_PSF_kr83m.h5',     
  e_cut           = 6e-3, #15 for 6mm  - 15 for 6mm
  n_iterations    = 100, #150 for 6mm  - 150 for 6mm
  iteration_tol   = 1e-10,
  sample_width    = [15.55, 15.55],
  bin_size        = [ 1,  1],
  diffusion       = (1., 0.2),
  n_dim           = 2,
  deconv_mode     = DeconvolutionMode.joint)#,
 # energy_type     = HitEnergy.Ec,
 # cut_type        = CutType.abs,
 # inter_method    = InterpolationMethod.cubic)
'''

# new parameters for threshold
pes = 1
threshold = thresh * pes
same_peak = True

deconv_params = dict(
                  drop_dist       = [16., 16.],
                  psf_fname       = f'/home/e78368jw/Documents/NEXT_CODE/Tl_studies/LPR/LightTables/NEXT100_PSF_kr83m_5bar.h5',
                  q_cut           = thresh,                   # WE HAVE MANUALLY CHANGED THIS FROM 10 -> 5 HERE
                  e_cut           = 9e-3,                # WE HAVE MANUALLY CHANGED THIS FROM 12e-3 TO 9e-3
                  n_iterations    = 75,
                  iteration_tol   = 1e-10,
                  sample_width    = [15.55, 15.55],
                  bin_size        = [ 1.,  1.],
                  satellite_iter  = 50,                       # after how many iterations do you start scanning? 
                  satellite_dist  = 2,                        # how far away do two clouds have to be to be considered unique
                  satellite_size  = 10,                       # maximum size of satellites, beyond which they are considered signal.
                  energy_type     = HitEnergy.Ec,               #Removing these variables because they were previously? 
                  diffusion       = (1.440, 1.440),
                  deconv_mode     = DeconvolutionMode.joint,
                  n_dim           = 2,
                  cut_type        = CutType.abs,              #Removing these variables because they were previously?
                  inter_method    = InterpolationMethod.cubic)            #Removing these variables because they were previously?  




#DeconvolutionMode
                
deconv_params_   = {k : v for k, v in deconv_params.items() if k not in ['q_cut', 'drop_dist']}

# Couple of functions used in beersheba.
cut_sensors       = beerfun.cut_over_Q   (deconv_params['q_cut'    ], ['E', 'Ec'])
drop_sensors      = beerfun.drop_isolated(deconv_params['drop_dist'], ['E', 'Ec'])

In [10]:
true_info = pd.read_hdf('Tl_signal_true_info.h5')
q = np.load('signal_ids.npy', allow_pickle = True)
MC_soph_data = pd.read_hdf('Tl_signal_soph.h5')

In [15]:
#if (datatype == 'MC'):
#    evtmap = mcio.load_eventnumbermap(soph_file).set_index('nexus_evt')
#    true_info = mcio.load_mchits_df(soph_file).reset_index()
#    true_info.event_id = true_info.event_id.map(evtmap.evt_number)



event_ids = []

# true track number
true_trk_number = []
time_spent = []
track_number = []

r_lim = 415
z_upper = 1170
z_lower = 20

file_number = file_no # alter this
p = 0
# collect the event from the full dataframe related to q

# this line applies the satellite killer
deconv_params_['satellite_iter'] = 25



for i in q:

    print("event {}".format(i))
        
    evt = i
    # signal events is the lower half
    dir_name = "plots/" + "event_" + str(i)


    if not os.path.exists(dir_name + '/iterations'):
        os.makedirs(dir_name + '/iterations', exist_ok = True)
    
    # check first that the MCof this event is within the 
    this_evt_true_info = true_info[true_info.event_id == i]

    xtMC = this_evt_true_info.x
    ytMC = this_evt_true_info.y
    ztMC = this_evt_true_info.z
    etMC = this_evt_true_info.energy*1000
    
    # Z
    z_max = ztMC.max()
    z_min = ztMC.min()
    
    # Radius
    r = np.sqrt(xtMC**2 + ytMC**2)
    r_max = r.max()

    if ((z_max > z_upper) or (z_min < z_lower) or (r_max > r_lim)):
        print("MC Max reached at: {}, {}, {}".format(z_max, z_min, r_max))
    else:

        hits = MC_soph_data[MC_soph_data.event == evt*2]
        #hits = hdst[hdst.event == evt]



        x_range, y_range, z_range, mid_x, mid_y, mid_z, min_x, min_y, min_z, max_x, max_y, max_z, xbins, ybins, zbins = collect_min_max_bins(hits)



        hits_cut = coref.timefunc(cut_sensors)(hits.copy())
        hits_drop = coref.timefunc(drop_sensors)(hits_cut.copy())

        # this is to stop your number of iterations vanishing because of the below function, as you're rewriting deconv_params_
        if 'save_state' in locals():
            deconv_params_['n_iterations'] = save_state
        else:
            # first time round, so set it
            save_state = deconv_params_['n_iterations']


        deconvolver = beerfun.deconvolve_signal(DataSiPM(detector_db, run_number), **deconv_params_)

        t0 = time.time()

        hits_deco = coref.timefunc(deconvolver)(hits_drop.copy())




        # Z
        z_max = hits_deco.Z.max()
        z_min = hits_deco.Z.min()

        # Radius
        r = np.sqrt(hits_deco.X**2 + hits_deco.Y**2)
        r_max = r.max()

        if ((z_max > z_upper) or (z_min < z_lower) or (r_max > r_lim)):

            print("RECO Max reached at: {}, {}, {}".format(z_max, z_min, r_max))
        else:



            time_spent.append(time.time() - t0)


            # plot the MC

                   # load in MC
            #evtmap = mcio.load_eventnumbermap(soph_file).set_index('nexus_evt')
            #true_info = mcio.load_mchits_df(soph_file).reset_index()
            #true_info.event_id = true_info.event_id.map(evtmap.evt_number)


            #this_evt_true_info = true_info[true_info.event_id == evt]

            #xtMC = this_evt_true_info.x
            #ytMC = this_evt_true_info.y
            #ztMC = this_evt_true_info.z
            #etMC = this_evt_true_info.energy*1000


            fog = plt.figure()
            ax = fog.add_subplot(111, projection='3d')

            p = ax.scatter(xtMC, ytMC, ztMC, alpha = 0.1, color='red')
            cb = fog.colorbar(p, ax=ax)
            cb.set_label('Energy (keV)')


            ax.set_xlabel('x (mm)')
            ax.set_ylabel('y (mm)')
            ax.set_zlabel('z (mm)')
            fog.suptitle('True Hits_3D' + dir_name, fontsize=30)

            fog.savefig(dir_name + '/MC_3D.png')
            plt.close(fog)

            # collect true track number here
            true_trk_number.append(count_tracks_mc(this_evt_true_info))



            # if its within fiducial, keep it
            event_ids.append(evt)


            x = hits_deco.X
            y = hits_deco.Y
            z = hits_deco.Z
            e = hits_deco.E

            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            p = ax.scatter(x, y, z, c=e, alpha = 0.07, cmap='viridis')
            cb = fig.colorbar(p, ax=ax)
            cb.set_label('Energy (keV)')

            ax.scatter(xtMC, ytMC, ztMC, color='red')

            ax.set_xlabel('x (mm)')
            ax.set_ylabel('y (mm)')
            ax.set_zlabel('z (mm)')

            fig.suptitle('3D_deconvolved_' + str(int(evt)) + "_iteration " + str(deconv_params_['n_iterations']), fontsize=30)
            #fig.savefig(pre_dir_name + dir_name +  "/3D_deconvolved_plot")
            fig.savefig(dir_name + '/iterations/3D_plot_' + str(deconv_params_['n_iterations']) + '_iter_MCinc.png')
            plt.close(fig)

            # stick track number on after finishing
            track_number.append(count_tracks(hits_deco))



event 140010
Time spent in cut_over_Q: 0.015415668487548828 s
Time spent in drop_isolated: 0.022321224212646484 s


/home/e78368jw/Documents/NEXT_CODE/next_misc/cluster_analysis_code/../../IC/invisible_cities/core/configure.py:235: UserWarning: Argument `satellite_iter` is not being used by `deconvolve_signal`
  warnings.warn(msg, UserWarning)
/home/e78368jw/Documents/NEXT_CODE/next_misc/cluster_analysis_code/../../IC/invisible_cities/core/configure.py:235: UserWarning: Argument `satellite_dist` is not being used by `deconvolve_signal`
  warnings.warn(msg, UserWarning)
/home/e78368jw/Documents/NEXT_CODE/next_misc/cluster_analysis_code/../../IC/invisible_cities/core/configure.py:235: UserWarning: Argument `satellite_size` is not being used by `deconvolve_signal`
  warnings.warn(msg, UserWarning)


TypeError: deconvolve_signal() got an unexpected keyword argument 'satellite_iter'

In [ ]:
np.save('time_spent_satkill' + str(file_no) + '.npy', np.array(time_spent))
np.save('track_num_satkill' + str(file_no) + '.npy', np.array(track_number))
np.save('true_trk_num' + str(file_no) + '.npy', np.array(true_trk_number))
np.save('eventids_satkill' + str(file_no) + '.npy', np.array(event_ids))

In [ ]:
if (datatype == 'MC'):
    evtmap = mcio.load_eventnumbermap(soph_file).set_index('nexus_evt')
    true_info = mcio.load_mchits_df(soph_file).reset_index()
    true_info.event_id = true_info.event_id.map(evtmap.evt_number)

# satellite killer
event_ids_nosat = []

r_lim = 415
z_upper = 1170
z_lower = 20





file_number = file_no # alter this
q = 0
cap = len(hdst.event.unique())

# this line applies the satellite killer
deconv_params_['satellite_iter'] = 9999

# true track number
true_trk_number_nosat = []
time_spent_nosat = []
track_number_nosat = []

for i in range(len(hdst.event.unique())):
    print("event {},{}/{}".format(hdst.event.unique()[i],q, cap))
    q += 1
    if (q == cap):
        break
        
    #evt = int(saved_signal_evts[i]*2)
    evt = hdst.event.unique()[i]
    # signal events is the lower half
    dir_name = "plots/" + "event_" + str(hdst.event.unique()[i]) + "_file_" + str(file_number)
    if not os.path.exists(dir_name + '/iterations_nosat'):
        os.makedirs(dir_name + '/iterations_nosat', exist_ok = True)
    

    
    
    
    # check first that the MCof this event is within the 
    this_evt_true_info = true_info[true_info.event_id == evt]

    xtMC = this_evt_true_info.x
    ytMC = this_evt_true_info.y
    ztMC = this_evt_true_info.z
    etMC = this_evt_true_info.energy*1000
    
    # Z
    z_max = ztMC.max()
    z_min = ztMC.min()
    
    # Radius
    r = np.sqrt(xtMC**2 + ytMC**2)
    r_max = r.max()

    if ((z_max > z_upper) or (z_min < z_lower) or (r_max > r_lim)):
        print("MC Max reached at: {}, {}, {}".format(z_max, z_min, r_max))
    else:
        hits = hdst[hdst.event == evt]



        x_range, y_range, z_range, mid_x, mid_y, mid_z, min_x, min_y, min_z, max_x, max_y, max_z, xbins, ybins, zbins = collect_min_max_bins(hits)



        hits_cut = coref.timefunc(cut_sensors)(hits.copy())
        hits_drop = coref.timefunc(drop_sensors)(hits_cut.copy())

        # this is to stop your number of iterations vanishing because of the below function, as you're rewriting deconv_params_
        if 'save_state' in locals():
            deconv_params_['n_iterations'] = save_state
        else:
            # first time round, so set it
            save_state = deconv_params_['n_iterations']

        iter_E = []
        iter_no = []
        i = 0
        p = 0


        deconvolver = beerfun.deconvolve_signal(DataSiPM(detector_db, run_number), **deconv_params_)

        t0 = time.time()

        hits_deco = coref.timefunc(deconvolver)(hits_drop.copy())




        # Z
        z_max = hits_deco.Z.max()
        z_min = hits_deco.Z.min()

        # Radius
        r = np.sqrt(hits_deco.X**2 + hits_deco.Y**2)
        r_max = r.max()

        if ((z_max > z_upper) or (z_min < z_lower) or (r_max > r_lim)):

            print("RECO Max reached at: {}, {}, {}".format(z_max, z_min, r_max))
        else:



            time_spent_nosat.append(time.time() - t0)


            # plot the MC

                   # load in MC
            evtmap = mcio.load_eventnumbermap(soph_file).set_index('nexus_evt')
            true_info = mcio.load_mchits_df(soph_file).reset_index()
            true_info.event_id = true_info.event_id.map(evtmap.evt_number)


            this_evt_true_info = true_info[true_info.event_id == evt]

            xtMC = this_evt_true_info.x
            ytMC = this_evt_true_info.y
            ztMC = this_evt_true_info.z
            etMC = this_evt_true_info.energy*1000


            fog = plt.figure()
            ax = fog.add_subplot(111, projection='3d')

            p = ax.scatter(xtMC, ytMC, ztMC, alpha = 0.1, color='red')
            cb = fog.colorbar(p, ax=ax)
            cb.set_label('Energy (keV)')


            ax.set_xlabel('x (mm)')
            ax.set_ylabel('y (mm)')
            ax.set_zlabel('z (mm)')
            fog.suptitle('True Hits_3D' + dir_name, fontsize=30)

            fog.savefig(dir_name + '/MC_3D.png')
            plt.close(fog)

            # collect true track number here
            true_trk_number_nosat.append(count_tracks_mc(this_evt_true_info))


            df_ps = pd.read_hdf(soph_file, 'MC/particles')
            df_ps = df_ps[df_ps.event_id == evt/2]

            #for pid, df in this_evt_true_info.groupby('particle_id'):
            this_evt_true_info['particle_name'] = this_evt_true_info['particle_id'].apply(return_id)

            fug, axus = plt.subplots(1, 3, figsize=(18, 4), constrained_layout=True)
            fug.suptitle('True Hits_' + dir_name, fontsize=30)
            #fig.tight_layout()
            added_labels = set()
            # Dictionary to store custom handles for the legend
            legend_handles = {}
            for pid, df in this_evt_true_info.groupby('particle_name'):

                xt = df.x
                yt = df.y
                zt = df.z
                et = df.energy*1000


                axus[0].scatter(xt, yt,  alpha=0.1, label = str(pid))
                axus[1].scatter(xt, zt,  alpha=0.1, label = str(pid))
                axus[2].scatter(yt, zt,  alpha=0.1, label = str(pid))


                axus[0].set_xlabel('X (mm)');
                axus[0].set_ylabel('Y (mm)');
                axus[1].set_xlabel('X (mm)');
                axus[1].set_ylabel('Z (mm)');
                axus[2].set_xlabel('Y (mm)');
                axus[2].set_ylabel('Z (mm)');

            # Retrieve legend handles and labels
            handles, labels = axus[0].get_legend_handles_labels()

            # Create a single legend for all subplots
            legend = fug.legend(handles, labels, fontsize=15, loc = 'lower left')
            for handle in legend.legendHandles:
                handle.set_alpha(1.0)

            fug.savefig(dir_name + '/MC_true.png')
            plt.close(fug)



            # if its within fiducial, keep it
            event_ids_nosat.append(evt)


            x = hits_deco.X
            y = hits_deco.Y
            z = hits_deco.Z
            e = hits_deco.E

            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            p = ax.scatter(x, y, z, c=e, alpha = 0.07, cmap='viridis')
            cb = fig.colorbar(p, ax=ax)
            cb.set_label('Energy (keV)')

            ax.scatter(xtMC, ytMC, ztMC, color='red')

            ax.set_xlabel('x (mm)')
            ax.set_ylabel('y (mm)')
            ax.set_zlabel('z (mm)')

            fig.suptitle('3D_deconvolved_' + str(int(evt)) + "_iteration " + str(deconv_params_['n_iterations']), fontsize=30)
            #fig.savefig(pre_dir_name + dir_name +  "/3D_deconvolved_plot")
            fig.savefig(dir_name + '/iterations_nosat/3D_plot_' + str(deconv_params_['n_iterations']) + '_iter_MCinc.png')
            plt.close(fig)

            # stick track number on after finishing
            track_number_nosat.append(count_tracks(hits_deco))



In [ ]:
np.save('time_spent_no_satkill' + str(file_no) + '.npy', np.array(time_spent_nosat))
np.save('track_num_no_satkill' + str(file_no) + '.npy', np.array(track_number_nosat))
np.save('true_trk_num_nosatkill' + str(file_no) + '.npy', np.array(true_trk_number_nosat))
np.save('eventids_nosat' + str(file_no) + '.npy', np.array(event_ids_nosat))

In [ ]:
# FIX THE RANGES
q = np.array([a - b for a, b in zip(track_number, true_trk_number)])
p = np.array([a - b for a, b in zip(track_number_nosat, true_trk_number_nosat)])
plt.hist(q, label = 'satellite killer', alpha = 0.5, bins=np.arange(-1,6)-0.5)
plt.hist(p, label = 'no satellite killer', alpha = 0.5, bins = np.arange(-1,10)-0.5)
plt.xticks(np.arange(-1, 11, 1.0))
plt.title('Reco tracks - true tracks: satellite killer - file ' + str(file_no))
plt.xlabel('Reco - true tracks')
plt.legend()
plt.ylabel('count')